follow from https://arxiv.org/abs/1701.05369, https://github.com/BayesWatch/tf-variational-dropout

（I am not confident to follow successfully... To review again...）

In [ ]:
# import os
# os.environ["CUDA_DEVICE_ORDER"]    = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())


In [ ]:
# import necessary libraries
import matplotlib
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

plt.ion()

from random import randint
import time

from ipywidgets import FloatProgress
from IPython.display import display, clear_output

import keras
from   keras.datasets   import mnist

from   tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

eps = 1e-10


In [ ]:
# data load
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape to 1 dim
X_train      = X_train.reshape(np.shape(X_train)[0], 
                               (np.shape(X_train)[1] * np.shape(X_train)[2]))
X_test       = X_test.reshape(np.shape(X_test)[0], 
                              (np.shape(X_test)[1] * np.shape(X_test)[2]))

# change type
X_train      = X_train.astype('float32')
X_test       = X_test.astype('float32')

# normalization
X_train_mean = np.mean(X_train)
X_train_std  = np.std(X_train)
X_train      = (X_train - X_train_mean) / (X_train_std + eps)
X_test       = (X_test  - X_train_mean) / (X_train_std + eps)

# convert class vectors to binary class matrices
class_num    = np.max(y_train) + 1 # consider class 0 
y_train      = keras.utils.to_categorical(y_train, class_num).astype('float32')
y_test       = keras.utils.to_categorical(y_test,  class_num).astype('float32')

print('np.shape(X_train) = (%d, %d)' % np.shape(X_train))
print('np.shape(y_train) = (%d, %d)' % np.shape(y_train))
print('np.shape(X_test)  = (%d, %d)' % np.shape(X_test))
print('np.shape(y_test)  = (%d, %d)' % np.shape(y_test))


In [ ]:
# set parameter for training
learning_rate         = 1e-3
epoch_num             = 50
mini_batch_size       = int(128 * 0.5)
mini_batch_iter       = 100 # if None then all train data use　on a epoch
random_seed           = 0

alpha___              = 0.1


In [ ]:
# fix mini batch iteration num
train_data_num      = len(y_train)
mini_batch_iter_tmp = round((train_data_num / mini_batch_size) - 0.5)
if (mini_batch_iter is None):
    mini_batch_iter = mini_batch_iter_tmp
else:
    if (mini_batch_iter > mini_batch_iter_tmp):
        mini_batch_iter = mini_batch_iter_tmp
        print('mini_batch_iter is too huge...')
        print('suppress to mini_batch_iter = %d' % mini_batch_iter)
        print('')

print('mini_batch_iter = %d' % mini_batch_iter)


In [ ]:
ard_init      = -10.0
thresh        = 3
activation_fn = tf.nn.relu

np.random.seed(random_seed)

sess          = tf.InteractiveSession()

# make model by tensorflow

# input layer
x             = tf.placeholder(dtype=tf.float32, shape=[None, 784])
phase         = tf.placeholder(dtype=tf.bool, shape=None)


# ---------- block 1 ----------

# define weight
weight_1      = tf.Variable(initial_value=tf.random_normal(shape=[784, 100], 
                                                          mean=0.0, stddev=0.01))
bias_1        = tf.Variable(initial_value=tf.zeros(shape=[100]))

# 
log_sigma2_1  = tf.Variable(initial_value=tf.constant(shape=[784, 100], 
                                                      value=ard_init))
log_alpha_1   = tf.clip_by_value(t=(log_sigma2_1 - tf.log(tf.square(weight_1) + eps)),
                                 clip_value_min=-8.0, clip_value_max=8.0)

# at train time, adding noise
mu_1          = tf.matmul(a=x, b=weight_1)
si_1          = tf.sqrt(x=tf.matmul(a=tf.square(x), 
                                    b=(tf.exp(log_alpha_1) * tf.square(weight_1))) + eps)
fc_noisy_1    = mu_1 + (si_1 * tf.random_normal(shape=tf.shape(mu_1), 
                                                mean=0.0, stddev=1.0))

# at test time, we just mask
select_mask_1 = tf.cast(x=tf.less(x=log_alpha_1, y=thresh), dtype=tf.float32)
fc_masked_1   = tf.matmul(a=x, b=(weight_1 * select_mask_1))

# choose between adding noise, or applying mask, depending on phase
matmul_1      = tf.cond(pred=phase, true_fn=(lambda:fc_noisy_1), false_fn=(lambda:fc_masked_1))
activations_1 = activation_fn(matmul_1 + bias_1)


# ---------- block 2 ----------

# define weight
weight_2      = tf.Variable(initial_value=tf.random_normal(shape=[100, 10], 
                                                          mean=0.0, stddev=0.01))
bias_2        = tf.Variable(initial_value=tf.zeros(shape=[10]))

# 
log_sigma2_2  = tf.Variable(initial_value=tf.constant(shape=[100, 10], 
                                                      value=ard_init))
log_alpha_2   = tf.clip_by_value(t=(log_sigma2_2 - tf.log(tf.square(weight_2) + eps)),
                                 clip_value_min=-8.0, clip_value_max=8.0)

# at train time, adding noise
mu_2          = tf.matmul(a=activations_1, b=weight_2)
si_2          = tf.sqrt(x=tf.matmul(a=tf.square(activations_1), 
                                    b=(tf.exp(log_alpha_2) * tf.square(weight_2))) + eps)
fc_noisy_2    = mu_2 + (si_2 * tf.random_normal(shape=tf.shape(mu_2), 
                                                mean=0.0, stddev=1.0))

# at test time, we just mask
select_mask_2 = tf.cast(x=tf.less(x=log_alpha_2, y=thresh), dtype=tf.float32)
fc_masked_2   = tf.matmul(a=activations_1, b=(weight_2 * select_mask_2))

# choose between adding noise, or applying mask, depending on phase
matmul_2      = tf.cond(pred=phase, true_fn=(lambda:fc_noisy_2), false_fn=(lambda:fc_masked_2))
activations_2 = activation_fn(matmul_2 + bias_2)


# output layer
y             = tf.nn.softmax(logits=activations_2)


In [ ]:
# define loss function and optimizer
y_            = tf.placeholder(dtype=tf.float32, shape=[None, 10])
prediction    = tf.clip_by_value(t=y, clip_value_min=eps, clip_value_max=1.0)
cross_entropy = - tf.reduce_mean(input_tensor=(y_ * tf.log(prediction)))

# prior DKL part of the ELBO
# utility to gather variational dropout parameters
k1, k2, k3    = 0.63576, 1.8732, 1.48695
C             = -k1

mdkl_1        = (k1 * tf.nn.sigmoid(k2 + k3 * log_alpha_1)) - (0.5 * tf.log1p(tf.exp(-log_alpha_1))) + C
dkl_qp_1      = -tf.reduce_sum(mdkl_1)
mdkl_2        = (k1 * tf.nn.sigmoid(k2 + k3 * log_alpha_2)) - (0.5 * tf.log1p(tf.exp(-log_alpha_2))) + C
dkl_qp_2      = -tf.reduce_sum(mdkl_2)

divergences   = []
divergences.append(dkl_qp_1)
divergences.append(dkl_qp_2)

# combine to form the ELBO
N             = float(np.shape(X_train)[0])
dkl           = tf.reduce_sum(tf.stack(divergences))
elbo          = cross_entropy + alpha___ * (1.0 / N) * dkl

# 
optimizer     = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step    = optimizer.minimize(loss=elbo)

tf.global_variables_initializer().run()


In [ ]:
# 
train_acc_stock  = np.zeros(epoch_num)
test_acc_stock   = np.zeros(epoch_num)
test_acc_best    = 0.0
train_loss_stock = np.zeros(epoch_num)
test_loss_stock  = np.zeros(epoch_num)

fig              = plt.figure(figsize=(12,25),dpi=100)
draw_pitch       = 5

# training
for epoch_i in range(epoch_num):
    
    # make index for mini batch selection
    rand_idx    = np.random.permutation(len(y_train))
    
    # mini batch iteration (attentive　SGD)
    for iter_i in range(mini_batch_iter):
        
        # set mini batch data
        X_train_mini_batch = X_train[rand_idx[:mini_batch_size], :]
        y_train_mini_batch = y_train[rand_idx[:mini_batch_size], :]
        rand_idx           = rand_idx[mini_batch_size:]
        
        # learning
        train_step.run(session=sess, 
                       feed_dict={x: X_train_mini_batch, 
                                  y_: y_train_mini_batch, 
                                  phase:True})
        
    # validaiton
    correct_prediction        = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    calc_acc                  = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    train_acc_stock[epoch_i]  = calc_acc.eval(feed_dict={x: X_train, y_: y_train, phase:False})
    test_acc_stock[epoch_i]   = calc_acc.eval(feed_dict={x: X_test,  y_: y_test,  phase:False})
    train_loss_stock[epoch_i] = sess.run(fetches=elbo, feed_dict={x: X_train, y_: y_train, phase:False})
    test_loss_stock[epoch_i]  = sess.run(fetches=elbo, feed_dict={x: X_test,  y_: y_test,  phase:False})
    
    # best record update
    if (test_acc_best < test_acc_stock[epoch_i]):
        test_acc_best = test_acc_stock[epoch_i]
    
    if (((epoch_i + 1) % draw_pitch) == 0):
        
        # get now state
        weight_1_tmp      = sess.run(weight_1)
        weight_1_tmp_sort = np.sort(weight_1_tmp.flat[:])
        log_alpha_1_tmp   = sess.run(log_alpha_1)
        fc_noisy_1_tmp    = sess.run(fc_noisy_1,  
                                     feed_dict={x:X_train_mini_batch, 
                                                weight_1:weight_1_tmp})
        fc_masked_1_tmp   = sess.run(fc_masked_1, 
                                     feed_dict={x:X_train_mini_batch, 
                                                weight_1:weight_1_tmp})
        
        fc_noisy_1_tmp_   = fc_noisy_1_tmp.copy()
        fc_masked_1_tmp_  = fc_masked_1_tmp.copy()
        fc_noisy_1_tmp_[fc_noisy_1_tmp_ < 0]   = 0 # relu
        fc_masked_1_tmp_[fc_masked_1_tmp_ < 0] = 0 # relu
        
        weight_2_tmp      = sess.run(weight_2)
        weight_2_tmp_sort = np.sort(weight_2_tmp.flat[:])
        log_alpha_2_tmp   = sess.run(log_alpha_2)
        fc_noisy_2_tmp    = sess.run(fc_noisy_2,  
                                     feed_dict={activations_1:fc_noisy_1_tmp_, 
                                                weight_2:weight_2_tmp})
        fc_masked_2_tmp   = sess.run(fc_masked_2, 
                                     feed_dict={activations_1:fc_masked_1_tmp_, 
                                                weight_2:weight_2_tmp})
        
        # draw
        ax_531 = fig.add_subplot(5, 3, 1)
        ax_531.cla()
        clear_output(wait=True)
        ax_531.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_531.imshow(weight_1_tmp, aspect='auto')
        # plt.colorbar()
        plt.title('imshow weight of 1st layer')
        display(fig)

        ax_532 = fig.add_subplot(5, 3, 2)
        ax_532.cla()
        clear_output(wait=True)
        ax_532.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_532.plot(weight_1_tmp_sort)
        plt.title('plot weight of 1st layer\n(sorted by value)')
        display(fig)

        ax_533 = fig.add_subplot(5, 3, 3)
        ax_533.cla()
        clear_output(wait=True)
        ax_533.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_533.hist(weight_1_tmp_sort, bins=50)
        plt.yscale('log')
        plt.title('histogram weight of 1st layer\n(std : %.3f)' % np.std(weight_1_tmp))
        display(fig)

        ax_534 = fig.add_subplot(5, 3, 4)
        ax_534.cla()
        clear_output(wait=True)
        ax_534.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_534.imshow(log_alpha_1_tmp, aspect='auto')
        # plt.colorbar()
        plt.title('imshow weight of 1st layer')
        display(fig)

        ax_535 = fig.add_subplot(5, 3, 5)
        ax_535.cla()
        clear_output(wait=True)
        ax_535.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_535.imshow(fc_noisy_1_tmp, aspect='auto')
        # plt.colorbar()
        plt.title('imshow noisy of 1st layer')
        display(fig)

        ax_536 = fig.add_subplot(5, 3, 6)
        ax_536.cla()
        clear_output(wait=True)
        ax_536.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_536.imshow(fc_masked_1_tmp, aspect='auto')
        # plt.colorbar()
        plt.title('imshow masked of 1st layer')
        display(fig)


        ax_537 = fig.add_subplot(5, 3, 7)
        ax_537.cla()
        clear_output(wait=True)
        ax_537.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_537.imshow(weight_2_tmp, aspect='auto')
        # plt.colorbar()
        plt.title('imshow weight of 2nd layer')
        display(fig)

        ax_538 = fig.add_subplot(5, 3, 8)
        ax_538.cla()
        clear_output(wait=True)
        ax_538.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_538.plot(weight_2_tmp_sort)
        plt.title('plot weight of 2nd layer\n(sorted by value)')
        display(fig)

        ax_539 = fig.add_subplot(5, 3, 9)
        ax_539.cla()
        clear_output(wait=True)
        ax_539.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_539.hist(weight_2_tmp_sort, bins=50)
        plt.yscale('log')
        plt.title('histogram weight of 2nd layer\n(std : %.3f)' % np.std(weight_1_tmp))
        display(fig)

        ax_53A = fig.add_subplot(5, 3, 10)
        ax_53A.cla()
        clear_output(wait=True)
        ax_53A.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_53A.imshow(log_alpha_2_tmp, aspect='auto')
        # plt.colorbar()
        plt.title('imshow weight of 2nd layer')
        display(fig)

        ax_53B = fig.add_subplot(5, 3, 11)
        ax_53B.cla()
        clear_output(wait=True)
        ax_53B.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_53B.imshow(fc_noisy_2_tmp, aspect='auto')
        # plt.colorbar()
        plt.title('imshow noisy of 2nd layer')
        display(fig)

        ax_53C = fig.add_subplot(5, 3, 12)
        ax_53C.cla()
        clear_output(wait=True)
        ax_53C.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_53C.imshow(fc_masked_2_tmp, aspect='auto')
        # plt.colorbar()
        plt.title('imshow masked of 2nd layer')
        display(fig)


        ax_529 = fig.add_subplot(5, 2, 9)
        ax_529.cla()
        clear_output(wait=True)
        ax_529.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_529.plot(train_acc_stock[:(epoch_i + 1)])
        ax_529.plot(test_acc_stock[:(epoch_i + 1)])
        plt.ylim(-0.05, 1.05)
        plt.title('accuracy of train data = %.2f%%\naccuracy of test data = %.2f%% (best : %.2f%%)' \
                  % ((train_acc_stock[epoch_i] * 100), (test_acc_stock[epoch_i] * 100), (test_acc_best * 100)))
        display(fig)

        ax_52A = fig.add_subplot(5, 2, 10)
        ax_52A.cla()
        clear_output(wait=True)
        ax_52A.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_52A.plot(train_loss_stock[:(epoch_i + 1)])
        ax_52A.plot(test_loss_stock[:(epoch_i + 1)])
        plt.ylim(-0.05, 1.05)
        plt.title('loss of train data = %.5f\nloss of test data = %.5f' \
                  % (train_loss_stock[epoch_i], test_loss_stock[epoch_i]))
        display(fig)
